Libraries

In [6]:
%matplotlib qt5
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LinearRegression

In [7]:
data = pd.read_csv("../Datasets/Salary_Data.csv")

X = np.array(data.iloc[:, 0])
y = np.array(data.iloc[:, 1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

def draw_test_and_train_data(ax):
    ax.scatter(X_train, y_train, label="Train", s=75, c="#1f77b4")
    ax.scatter(X_test, y_test, label="Test", marker="x", s=75, c="#1f77b4")
    ax.set_xlabel("Years experience", fontsize=14)
    ax.set_ylabel("Salary", fontsize=14)
    ax.legend()


In [8]:

def ssr(m, b):
    sum = 0
    for i in range(len(X)):
      sum += (y[i] - (m * X[i] + b) ) ** 2
    
    return sum

def draw_prediction_line(ax, min, max, m, b):
    x = np.arange(min-1, max+1)
    y =  m * x + b
    ax.plot(x, y)

In [9]:
class LinearRegressionModel():
    def __init__(self):
        self.X = None
        self.y = None

    def ssr_gradient(self, m, b):
        sum_m = 0
        sum_b = 0
        n = len(self.X)
        for i in range(n):
            error = self.y[i] - (m * self.X[i] + b)
            derivative_m = -2 * self.X[i] * error  # Derivative w.r.t. m
            derivative_b = -2 * error              # Derivative w.r.t. b
            sum_m += derivative_m
            sum_b += derivative_b

        return sum_m, sum_b
    
    def fit(self, X, y, m, b): # Gradient Descent
        self.X = X
        self.y = y

        current_pos = (m, b)
        learning_rate = 0.001
        min_step_size = 0.001
        max_steps = 10000
        current_steps = 0

        while(current_steps < max_steps):
            M_derivative, B_derivative = self.ssr_gradient(current_pos[0], current_pos[1])
            M_step_size, B_step_size = M_derivative * learning_rate, B_derivative * learning_rate

            if  abs(M_step_size) < min_step_size and abs(B_step_size) < min_step_size:
                break

            M_new, B_new = current_pos[0] - M_step_size, current_pos[1] - B_step_size
            
            current_pos = (M_new, B_new)

            print(f"Parameters: m: {current_pos[0]}; b: {current_pos[1]}")
            print(f"Epoch: {current_steps}")
            print(f"SSR: {ssr(current_pos[0], current_pos[1])}")

            current_steps += 1
        
        self.m = current_pos[0]
        self.b = current_pos[1]

    def predict(self, X_test):
        return self.m * X_test + self.b

In [10]:
lin_reg_model = LinearRegressionModel()
lin_reg_model.fit(X_train, y_train, 9911, 9221)

sklearn_lin_reg_model = LinearRegression()
sklearn_lin_reg_model.fit(X_train.reshape(-1, 1), y_train)

plt.close("all")
fig, (ax, ax_sklearn) = plt.subplots(1, 2, figsize=(10, 5))
ax.set_title(f"Own Lin-Reg-model")
ax_sklearn.set_title("Sklearn Lin-Reg-model")

draw_test_and_train_data(ax)
draw_test_and_train_data(ax_sklearn)

m_own = round(lin_reg_model.m, 2)
b_own = round(lin_reg_model.b, 2)
m_sklearn = round(sklearn_lin_reg_model.coef_[0], 2)
b_sklearn = round(sklearn_lin_reg_model.intercept_, 2)
ssr_own = round(ssr(m_own, b_own), 2)
ssr_sklearn = round(ssr(m_sklearn, b_sklearn), 2)

draw_prediction_line(ax, np.min(X), np.max(X), m_own, b_own)
draw_prediction_line(ax_sklearn, np.min(X), np.max(X), m_sklearn, b_sklearn)

for years_experience, salary in zip(X_test, y_test):
    predicted_salary = lin_reg_model.predict(years_experience)
    #print(f"Experience: {years_experience} years - Salary: {salary} - Predicted salary: {predicted_salary}")
    ax.plot([years_experience, years_experience], [salary, predicted_salary], lw=1, c="red")

    sklearn_predicted_salary = sklearn_lin_reg_model.predict(years_experience.reshape(-1, 1))[0]
    #print(f"Experience: {years_experience} years - Salary: {salary} - Predicted salary: {sklearn_predicted_salary}")
    ax_sklearn.plot([years_experience, years_experience], [salary, sklearn_predicted_salary], lw=1, c="red")


print(f"Own ssr = {ssr_own} with m = {m_own} and b = {b_own}")
print(f"Sklearn ssr = {ssr_sklearn} with m = {m_sklearn} and b = {b_sklearn}")
print(f"Sklearn score {sklearn_lin_reg_model.score(X_test.reshape(-1, 1), y_test)}")


Parameters: m: 12083.62062; b: 9679.2406
Epoch: 0
SSR: 2692862868.853629
Parameters: m: 12082.5067330404; b: 9794.799220132
Epoch: 1
SSR: 2677910831.592652
Parameters: m: 12064.976159624144; b: 9907.059724145724
Epoch: 2
SSR: 2654131443.6468353
Parameters: m: 12047.429108699727; b: 10018.606541854257
Epoch: 3
SSR: 2630592340.003913
Parameters: m: 12029.98959215861; b: 10129.46357854149
Epoch: 4
SSR: 2607360658.6797714
Parameters: m: 12012.657887194442; b: 10239.635246932432
Epoch: 5
SSR: 2584433031.987105
Parameters: m: 11995.43333443008; b: 10349.125785403221
Epoch: 6
SSR: 2561805606.610276
Parameters: m: 11978.315271457746; b: 10457.93940500841
Epoch: 7
SSR: 2539474573.411019
Parameters: m: 11961.303039910928; b: 10566.080290759455
Epoch: 8
SSR: 2517436170.846711
Parameters: m: 11944.39598549302; b: 10673.552601794157
Epoch: 9
SSR: 2495686684.409578
Parameters: m: 11927.593457952556; b: 10780.36047153669
Epoch: 10
SSR: 2474222446.042992
Parameters: m: 11910.894811058215; b: 10886.508

Linear regression class